In [1]:
import itertools

import numpy as np
import pandas as pd
from numpy.linalg import LinAlgError
from scipy.stats import boxcox
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA

# ARIMA model

We will fit ARIMA model using water_level in Dusseldorf, sequence by sequence.

## Reading and visualising the data
The station in Dusseldorf is #6335050

In [2]:
river_df = pd.read_csv('./stations/station_6335050_river_data.csv')
river_df.date = pd.to_datetime(river_df.date, format='%Y-%m-%d')
river_df = river_df.set_index('date')

FileNotFoundError: [Errno 2] File b'../stations/station_6335050_river_data.csv' does not exist: b'../stations/station_6335050_river_data.csv'

In [ ]:
river_df.delta1.plot()

## Function for fitting one sequence of water_level

Our data is clearly non-stationary (see river_data_exploration_final_amina.ipynb)  
so we will use Box-Cox transform to ensure stationarity.  

Only then we can try to fit ARIMA.  

At the end we will apply reverse Box-Cox transformation to get the output data in initial form.  

The parameters are:
df - the pandas dataframe with water_level column from river_data.csv;  
the gaps (dates without any data) should be filled by NaNs
pred_start - the first day after the sequence, it signifies the first day of the gap, the first day we should predict
pred_end - the last day, that we should fill with our predictions

In [12]:
MAX_PARAM_COUNT = 8

def fit_and_predict(df, pred_start, pred_end):
    head_df = df[df.index < pred_start]
    train, val = train_test_split(head_df.water_level.values, test_size=0.2, shuffle=False)
    
    norm, lambd = boxcox(train)
    train = (train**lambd - 1)/lambd
    
    p = q = range(0, min(MAX_PARAM_COUNT, int(np.sqrt(train.shape[0]))))
    d = range(1, 2)
    pdq = list(itertools.product(p, d, q))
    best_r2 = -np.inf
    best_params = pdq[0]
    best_arima = None
    
    for (p, d, q) in pdq:
        if p == 0 and d == 0 and q == 0: continue
        
        arima = ARIMA(train, (p, d, q))
        try:
            arima_fit = arima.fit()
        except LinAlgError:
            continue
        except ValueError:
            continue
            
        if d == 1:
            pred = arima_fit.forecast(len(val))[0]
        else:
            pred = arima_fit.forecast(len(val))[0]

        try:
            r2 = r2_score(val, pred)
        except ValueError:
            print(f'Flawed prediction for {p}, {d}, {q}')
            continue
            
        if r2 > best_r2:
            best_r2 = r2
            best_params = (p, d, q)
            best_arima = arima_fit
        
    print('Best r2:', best_r2)
    print('Best params:', best_params)
    norm, lambd = boxcox(head_df.water_level.values)
    print(f'Best params: {best_params}')
    
    arima = ARIMA(norm, best_params).fit()

    if best_params[1] == 1:
        pred_fin = arima.forecast(17)[0]
    else:
        pred_fin = best_arima.forecast(17)[0]
    
    pred_dates = pd.date_range(pred_start, pred_end).to_series().reset_index(drop=True)
        
    def invboxcox(y):
          return ((y*lambd)+1)**(1/lambd)
    
    df.loc[pred_dates, 'water_level'] = invboxcox(pred_fin)

## Setting up our dataset and feeding it to ARIMA

In [ ]:
full_range = pd.date_range(river_df.index.min(), river_df.index.max()+pd.Timedelta(days=1))
river_df = river_df.reindex(full_range, fill_value=np.NaN)

next_gap = river_df.water_level.isnull().idxmax()
tail_gap = next_gap + pd.Timedelta(days=16)

while next_gap < pd.to_datetime('2013-01-01'):
    print(f'Working with gap {next_gap} - {tail_gap}.')
    fit_and_predict(river_df, next_gap, tail_gap)
    river_df[river_df.index <= tail_gap].water_level.plot()
    
    next_gap = river_df.water_level.isnull().idxmax()
    tail_gap = next_gap + pd.Timedelta(days=16)

## Saving the results

In [ ]:
river_df[['water_level']].to_csv('station6335050_water_level.csv')

## Checking the results

In [ ]:
river_df[100:110]

In [ ]:
river_df.water_level.plot()